# Yolov8 object detection

### Local test

In [ ]:
%pip install ultralytics

In [ ]:
from ultralytics import YOLO
from PIL import Image

model = YOLO('yolov8n.pt')
model.to('cuda')

In [ ]:
results = model(['image.png', 'bus.jpg'], device='cuda')

In [ ]:
# Show the results
for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    im.show()  # show image
    # im.save('result.jpg')  # save image

### Deploy on SageMaker

In [ ]:
# init sagemaker
import boto3
import sagemaker
from sagemaker import serializers, deserializers
from sagemaker.pytorch.model import PyTorchModel, PyTorchPredictor

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts
region = sess._region_name  # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment

s3_model_prefix = "yolov8-deployment"

print(f"role: {role}")
print(f"bucket: {bucket}")

In [ ]:
!mkdir mymodel
!cp yolov8n.pt mymodel/

In [ ]:
!rm -f model.tar.gz
!rm -rf mymodel/.ipynb_checkpoints
!tar czvf model.tar.gz -C mymodel .
s3_model_artifact = sess.upload_data("model.tar.gz", bucket, s3_model_prefix)
print(f"S3 Code or Model tar uploaded to --- > {s3_model_artifact}")

In [ ]:
# deployment
framework_version = '1.13'
py_version = 'py39'
instance_type = "ml.g4dn.xlarge"
endpoint_name = sagemaker.utils.name_from_base("yolov8")

model = PyTorchModel(
    model_data = s3_model_artifact,
    entry_point = 'inference.py',
    source_dir = "./code/",
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
)

model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=endpoint_name,
)

# our requests and responses will be in json format so we specify the serializer and the deserializer
predictor = PyTorchPredictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
    serializer=serializers.JSONSerializer(),
    deserializer=deserializers.JSONDeserializer(),
)

In [ ]:
import io
import base64

"""
import json

res = {'results': []}

for r in results:
    item = {}
    item['objects'] = json.loads(r.tojson())
    
    im_array = r.plot()  
    im = Image.fromarray(im_array[..., ::-1])
    byteImgIO = io.BytesIO()
    im.save(byteImgIO, "WEBP")
    byteImgIO.seek(0)
    byteImg = byteImgIO.read()
    imgstr = base64.b64encode(byteImg).decode('ascii')
    item['image'] = imgstr
    
    res['results'].append(item)

json.dumps(res)
"""

img_lst = ['bus.jpg', 'image.png']

def predict_fn(predictor, img_lst):
    input_data = {'input_images': []}

    for i in img_lst:
        img = Image.open(i)
        byteImgIO = io.BytesIO()
        img.save(byteImgIO, "PNG")
        byteImgIO.seek(0)
        byteImg = byteImgIO.read()
        imgstr = base64.b64encode(byteImg).decode('ascii')
        input_data['input_images'].append(imgstr)
    
    res = predictor.predict(input_data)

    for item in res['results']:
        objects = item['objects']
        image = item['image']
        dataBytesIO = io.BytesIO(base64.b64decode(image))
        image = Image.open(dataBytesIO)
        print(objects)
        image.show()

In [ ]:
# predict
predict_fn(predictor, img_lst)